In [51]:
import cv2
import numpy as np
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool

#Initialiing pools for parallelizing the tasks
cores=mp.cpu_count()
pool = Pool(cores)


#GLOBAL VARIABLES
count = 0

def denoising(img):
    global count
    
    
    # SET THE THRESHOLD HERE
    threshold=5
    
    
    bool_array=img.copy()
    img_copy = img.copy()
    x = 0
    
    # To check whether an island is within the threshold limit
    def dfs(image, i, j, threshold):
        global count
        if i < 0 or j < 0 or i >= image.shape[0] or j >= image.shape[1] or image[i, j] == 0:
            return
        if count > threshold:
            return
        if image[i, j] == 255:
            count += 1
            image[i, j] = 0
            dfs(image, i + 1, j, threshold)
            dfs(image, i, j + 1, threshold)
            dfs(image, i, j - 1, threshold)
            dfs(image, i - 1, j, threshold)
        else:
            return
        
    # To mask the islands which are above the threshold limit
    # Mainly for optimising the code
    def dfs_convert(i, j,bool_arrays):
        if i < 0 or j < 0 or i >= bool_arrays.shape[0] or j >= bool_arrays.shape[1] or bool_arrays[i, j] == 0:
            return
        bool_arrays[i][j] = 0
        dfs_convert(i + 1, j,bool_arrays)
        dfs_convert(i, j + 1,bool_arrays)
        dfs_convert(i, j - 1,bool_arrays)
        dfs_convert(i - 1, j,bool_arrays)
        return bool_arrays
    
    #Visiting each pixels
    for k in range(bool_array.shape[0] ):
        for l in range(bool_array.shape[1] ):
            if img[k, l] == 255 and bool_array[k, l] == 255:
                count = 0
                dfs(img_copy.copy(), k, l, threshold)
                if (count > threshold):
                    x += 1
                    bool_array=dfs_convert(k, l,bool_array)
    return bool_array

def start_denoise(image_input):
    print("***Denoising Started**")
    min_height=400  #min-width and min-height of one frame
    min_width=400   #Set to optimal values after lots of testing
    width=image_input.shape[1]
    height=image_input.shape[0]
    df_out=np.ones((0,width))
    if(width>=min_width and height>=min_height):
        print("***Denoising via parallelizing**")
        no_columns=int(width/min_width)
        no_rows=int(height/min_height)
    
        df_split = np.array_split(image_input,no_rows, axis=0)        
        print(df_out.shape)
    
        for i in df_split:
            df_out1=np.ones((i.shape[0],0))
            df=np.array_split(i,cores,axis=1)
            df_out1 = np.hstack(pool.map(denoising,df))
            df_out=np.vstack((df_out,df_out1))
            print(df_out1.shape)
    else:
        print("***Denoising without parallelizing**")
        df_out=denoising(image_input)
    return df_out
        
# Variables for Timer
total_time = 0.0
start_timer = 0.0
end_timer = 0.0

# START TIMER
start_timer = cv2.getTickCount()

#INPUT
eg = cv2.imread("C:/Users/bhuva/Downloads/Untitled Folder/testing/test_input_image.png", cv2.IMREAD_GRAYSCALE);
# eg = eg[0:600, 0:600]


#Calling the function
output_noise=start_denoise(eg)


# END TIMER
end_timer = cv2.getTickCount()


# Restoring the image without noise and saving it
res = eg.copy()
for a in range(len(output_noise)):
    for b in range(len(output_noise[0])):
        if output_noise[a, b] == 255:
            res[a, b] = 0
cv2.imwrite('C:/Users/bhuva/Downloads/Untitled Folder/testing/result1.png',res)

# Printing the Timer results
message = 'Time taken: {:0.4f} ms'
total_time = (end_timer-start_timer)*1000/cv2.getTickFrequency()
print(message.format(total_time))

# Checking the accuracy
result=cv2.imread("C:/Users/bhuva/Downloads/Untitled Folder/testing/result1.png", cv2.IMREAD_GRAYSCALE);
image_binary = cv2.imread("C:/Users/bhuva/Downloads/Untitled Folder/testing/test_input_image.png", cv2.IMREAD_GRAYSCALE);
loss=0
removed=0
for i in range(len(image_binary)):
    for j in range(len(image_binary[0])):
        if image_binary[i,j]==255 and result[i,j]!=255:
            removed+=1
        if image_binary[i,j]==0 and result[i,j]==255:
            loss+=1    
print("Loss :",str(loss)," pixels lost")
print("Removed :",str(removed)," pixels removed")


# Shutting the pools
pool.close()
pool.join()
pool.clear()

***Denoising Started**
***Denoising without parallelizing**
Time taken: 158.4092 ms
Loss : 0  pixels lost
Removed : 666  pixels removed
